In [58]:
import hashlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [59]:
class Attendance:
    """
    Extracts data given a directory containing reference and transient datasets as .txt files.
    
    :param string directory:
        The directory in which all of the recorded datasets are stored.
        Default is "../streams".
    :param string reference_file:
        The name of the file intended to be used as the reference dataset.
        Default is "podium.txt".
    """
    def __init__(self, directory="streams", reference_file="podium.txt"):
        self.directory = directory
        self.reference_file = reference_file
        self.reference_data = self.dictify_data(self.reference_file)
        self.operational_data = self.dictify_op_data()
        self.location_scores = self.calc_loc_scores()
        self.matrix_data = self.gen_matrix_data() # Stores y_true and y_pred
        self.con_matrix = confusion_matrix(self.matrix_data[0], self.matrix_data[1])
        self.class_report = classification_report(self.matrix_data[0], self.matrix_data[1],
                                                  target_names=['Outside', 'Inside'])
     
    
    def dictify_data(self, file):
        """
        Called by constructor to create reference data attribute.
        Pandas Documentation: https://pandas.pydata.org/pandas-docs/stable/reference/index.html
        
        :param string file:
            The name of the file that contains data to be cleaned.
            Must be inside of the object's directory
            e.g. "podium.txt" or "first_row.txt".

        :return:
            A dictionary containing MAC addresses as keys and their respective received signal 
            strength indicators as values. {"a1:b2:c3:d4:e5:f6": 0, ... "f6:e5:d4:c3:b2:a1": -95}
        :rtype: dict
        """
            
        # use the object's directory attribute, and function parameter for file
        file_path = f"{self.directory}/{file}"
        # convert the text data into a dataframe, while preserving first row
        df = pd.read_csv(file_path, header=None)
        df = df.iloc[:, 3:-1] # drop device id, latitude, longitude & date columns
        df = pd.concat([ # enter hacky list comprehension pandas concatenation
            df.iloc[:, col:col+2].rename(columns={ # index mac address and rssi column pairs
                col+3: "mac_address", # index for the current mac address column
                col+4: "average_rssi" # index for the current rssi column
            }) # the column indexes from the result previous iloc, starting at 3,
            # the columns could be aligned with some transposes and resetting the index instead
            for col in range(0, len(df.columns), 2) # perform concatenation for every pair
        ], ignore_index=True)

        df.replace(0, -95, inplace=True) # replace all zero values with the -95 sensitivity
        # update mac_address column to hash addresses with md5 and temporarily add dummy salt to hash
        df["mac_address"] = [hashlib.md5((str(val) + "salt").encode("UTF-8")).hexdigest()
                             for val in df["mac_address"]]
        # group by mac address and calculate the average rssi for each unique mac address
        df = df.groupby("mac_address", as_index=False)["average_rssi"].mean()
        
        try: # try except since attribute will not exist if reference data hasn't been dictified
            if self.reference_data: # only enters block for operational data
                # select only indexes where mac address is in reference dictionary keys
                df = df[df["mac_address"].isin(self.reference_data)]
                if len(df.index) != len(self.reference_data): # enters block if any indexes
                    indexes = []                              # were missing from dataframe
                    for mac in self.reference_data: # check which mac address is missing
                        if not df["mac_address"].str.contains(mac).any(): # if missing
                            indexes.append([mac, -95]) # add corresponding mac and sensitivity
                    # append newly created dataframe from missing indexes to complete it
                    df = df.append(pd.DataFrame(indexes, columns=["mac_address", "average_rssi"]))
        except:
            pass # no handling needed
        
        # sort by highest average rssi
        df.sort_values(by="average_rssi", ascending=False, inplace=True, ignore_index=True)
        df = df.head(10) # select the top 20 indexes after sorted
        
        # set the index column to the mac address and select the average rssi column,
        # allowing to_dict() to return in {mac_address: average_rssi} format
        dictified = df.set_index("mac_address")["average_rssi"].to_dict()
        
        return dictified # return completed dictionary


    def dictify_op_data(self):
        """
        Called by constructor to create the operational data attribute.
        OS Documentation: https://docs.python.org/3/library/os.html
        Pandas Documentation: https://pandas.pydata.org/pandas-docs/stable/reference/index.html
        
        :return:
            A 2-D dictionary containing file names as keys with dictionaries containing 
            MAC addresses and their respective received signal strength indicators as values. 
            {"file_1.txt": {"a1:b2:c3:d4:e5:f6": 0,  ... "f6:e5:d4:c3:b2:a1": -95}, ...
             "file_n.txt": {"b2:c3:d4:e5:f6:a1": -45,  ... "c3:d4:e5:f6:a1:b2": -90}}
        :rtype: dict
        """
        
        dictified = {}
        
        for root, subdirs, files in os.walk(self.directory): # walk through all subdirs and files
            for file in files: # apply to all files found in the walkthrough
                # select only txt files and not the specified reference file.
                if file.endswith(".txt") and file != self.reference_file: 
                    file_path = os.path.join(root, file) # full file path including the root
                    # obtain relative path by removing root directory
                    rel_path = os.path.relpath(file_path, self.directory)
                    # convert the text data into a dataframe, while preserving first row
                    df = pd.read_csv(file_path, header=None)
                    # create new dictionary key value pair matching format
                    # {rel_path: {mac_address: average_rssi}}
                    dictified[rel_path] = self.dictify_data(rel_path)
                
        return dictified # return completed dictionary


    def calc_loc_scores(self):
        """
        Called by constructor to create the location scores attribute.
        Calculates the Euclidian Distance between reference and operational data.
        NumPy Documentation: https://numpy.org/doc/stable/reference/index.html
        
        :return:
            A dictionary containing file names as keys and their respective calculated
            euclidian distances as values. {"file_1.txt": 50.125, ... "file_n": 93.725}
        :rtype: dict
        """
        
        location_scores = {}

        # create numpy array with list comprehension, sorting keys, then adding values to list
        ref = np.array([self.reference_data[mac] for mac in sorted(self.reference_data)])
        for file, ms_pair in self.operational_data.items(): # iterate through op data dictionary
            # create numpy array with list comp, for n'th operational dataset to compare with ref
            op = np.array([ms_pair[mac] for mac in sorted(ms_pair)])
            # calculate euclidian distance with numpy's euclidian norm method
            euclidist = np.linalg.norm(ref - op)
            # add entry to return dictionary in format {file_name: euclidian_distance}
            location_scores[file] = euclidist
        
        return location_scores # return completed dictionary
    
    
    def gen_matrix_data(self):
        """
        Iterate through dicts in self.operational_data. For each rssi value, compare to threshold and 
        make a prediction which is added to y_pred list. Using the file prefix, determine whether 
        inside or outside and add to y_true list. 
        Sklearn Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
        
        :return: 
            A tuple containing 2 lists, y_true and y_pred.
            
        :rtype: tuple
        """
        y_true = []
        y_pred = []
        threshold = 56.5   # The average RSSI threshold used to decide if in or out of room
        for data_set in self.operational_data:
            for rssi in self.operational_data[data_set].values():
                rssi = rssi * -1 # Make positive 
                # print(data_set[0], rssi)
                # Check whether the data set file prefix indicates inside or outside the room.  
                if data_set[0] == "i":
                    y_true.append('inside')
                else:
                    y_true.append('outside')

                # Compare the average RSSI of each access point to the threshold.
                if rssi < threshold: 
                    y_pred.append('inside')
                else:
                    y_pred.append('outside')
        return y_true, y_pred


    def disp_con_matrix(self):
        labels = ["Outside", "Inside"]
        disp = ConfusionMatrixDisplay(confusion_matrix=self.con_matrix, display_labels=labels)
        print("n =", len(self.matrix_data[1]))
        disp.plot(cmap='Blues')

In [60]:
att = Attendance("./input", "instructor.txt")

In [61]:
att.reference_data

{'40b85cbe81fa287decda2cc95cf25d6c': -58.84705882352941,
 '1eeafe7256f20b52ee0c03b9a7616781': -59.15294117647059,
 '289c7e5dbf8ff268a306ee369fca571a': -59.21176470588235,
 '4b75bd065f72be9f557d3efc17e303b8': -59.305882352941175,
 '918a1f575bbe128cf541052934588c2c': -61.23529411764706,
 '6609a096aa3508b99078b8d66c65c695': -61.55294117647059,
 '33573f24fc64a924690e3f4b7af5974b': -61.65882352941176,
 '8786f78a4070c4027e9704f21505fb77': -61.67058823529412,
 'e6cc847ebc6205cfb9376d3c3f21c2e8': -61.68235294117647,
 '159debbfa57e5a802e2cbcd280c8111c': -61.694117647058825}

In [62]:
att.operational_data

{'inside/second_row.txt': {'6609a096aa3508b99078b8d66c65c695': -43.285714285714285,
  '8786f78a4070c4027e9704f21505fb77': -43.642857142857146,
  '33573f24fc64a924690e3f4b7af5974b': -43.714285714285715,
  'e6cc847ebc6205cfb9376d3c3f21c2e8': -43.714285714285715,
  '159debbfa57e5a802e2cbcd280c8111c': -44.0,
  '40b85cbe81fa287decda2cc95cf25d6c': -45.0,
  '1eeafe7256f20b52ee0c03b9a7616781': -45.07142857142857,
  '289c7e5dbf8ff268a306ee369fca571a': -45.07142857142857,
  '4b75bd065f72be9f557d3efc17e303b8': -45.07142857142857,
  '918a1f575bbe128cf541052934588c2c': -45.785714285714285},
 'inside/third_row_right.txt': {'6609a096aa3508b99078b8d66c65c695': -48.7,
  '159debbfa57e5a802e2cbcd280c8111c': -49.4,
  '33573f24fc64a924690e3f4b7af5974b': -49.4,
  '8786f78a4070c4027e9704f21505fb77': -49.4,
  'e6cc847ebc6205cfb9376d3c3f21c2e8': -49.4,
  '40b85cbe81fa287decda2cc95cf25d6c': -50.6,
  '1eeafe7256f20b52ee0c03b9a7616781': -50.7,
  '289c7e5dbf8ff268a306ee369fca571a': -50.7,
  '4b75bd065f72be9f557d3e

In [63]:
att.location_scores

{'inside/second_row.txt': 51.45793112767909,
 'inside/third_row_right.txt': 34.13232594975743,
 'inside/fourth_row_right.txt': 18.80473727219519,
 'inside/fourth_row_left.txt': 30.676362303119465,
 'inside/room_center.txt': 33.64711667216964,
 'inside/third_row_left.txt': 44.929758786024365,
 'inside/first_row_right.txt': 29.480405708794162,
 'inside/first_row_left.txt': 36.08468266810368,
 'outside/at_the_door.txt': 7.4058957856253445,
 'outside/classroom_2044.txt': 93.00302686471086,
 'outside/cs_dept.txt': 92.65659146933012,
 'outside/classroom_2003.txt': 99.3134216714617,
 'outside/study_room.txt': 97.79314723092166,
 'outside/red_chairs.txt': 52.00954996699324}

In [64]:
att.matrix_data

(['inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'inside',
  'outside',
  'outside',
  'outside',
 

In [65]:
att.con_matrix

array([[79,  1],
       [ 0, 60]])

In [57]:
att.class_report

'              precision    recall  f1-score   support\n\n     Outside       1.00      0.99      0.99        80\n      Inside       0.98      1.00      0.99        60\n\n    accuracy                           0.99       140\n   macro avg       0.99      0.99      0.99       140\nweighted avg       0.99      0.99      0.99       140\n'